# Project - Decision Tree

This data set for this exercise is from healthcare. It contains data about 10 years of clinical care at 130 US Hospitals. Each row represents a single patient. The columns include the characteristics of deidentified diabetes patients. This is a binary classification task: predict whether a diabetes patient is readmitted to the hospital within 30 days of their discharge (1=Yes, 0=No). This is an important performance metric for hospitals as they try to minimize these types of readmissions.

## Description of Variables

The description of variables are provided in "Healthcare (small) - Data Dictionary.docx"

## Goal

Use the **healthcare_small.csv** data set and build a model to predict **readmitted**. 

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

diabetes = pd.read_csv("healthcare_small.csv")
diabetes.head()

,race,gender,age,admission_type,discharge_disposition,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
0,Other,Female,70-80,2,3,1,14,32,3,15,0,0,0,9,1
1,Caucasian,Female,80-90,1,3,5,4,44,0,15,0,0,0,9,0
2,AfricanAmerican,Male,50-60,5,1,1,6,29,1,15,0,0,0,9,1
3,Caucasian,Female,50-60,1,1,6,3,47,0,10,0,0,0,4,0
4,AfricanAmerican,Female,40-50,3,1,1,4,92,0,15,0,0,0,7,0


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train_set, test = train_test_split(diabetes, test_size=0.3)

# Data Prep

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train_set['readmitted']
test_target = test['readmitted']

train_inputs = train_set.drop(['readmitted'], axis=1)
test_inputs = test.drop(['readmitted'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Whether the patient had any emergency visits or not
- Ratio of inpatient visits to outpatient visits
- Ratio of time in hospital to number of medications
- (etc.)

In [6]:
def new_col(df):
    
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()

    # Use the formula, though fill in 0s when the value is 0/0 (because 0/0 generates "nan" values)
    df1['lab_tests_per_procedure'] = (df1['num_lab_procedures']/df1['num_procedures']).fillna(0)

    # Replace the infinity values with 1 (because a value divided by 0 generates infinity)
    df1['lab_tests_per_procedure'].replace(np.inf, 1, inplace=True)

    return df1[['lab_tests_per_procedure']]
    # Using this to check whether the calculation is made correctly:
    #return df1
    

In [7]:
#Let's test the new function:

# Send the train set to the function we created
new_col(train_set)


,lab_tests_per_procedure
4166,39.0
5546,1.0
2957,20.5
6329,54.0
565,7.5
...,...
5734,11.0
5191,1.0
5390,1.0
860,6.2


##  Identify the numeric, binary, and categorical columns

In [8]:
# Let's change some of the categorical variables' data type to object

train_inputs['admission_source'] = train_inputs['admission_source'].astype('object')
train_inputs['admission_type'] = train_inputs['admission_type'].astype('object')
train_inputs['discharge_disposition'] = train_inputs['discharge_disposition'].astype('object')

test_inputs['admission_source'] = test_inputs['admission_source'].astype('object')
test_inputs['admission_type'] = test_inputs['admission_type'].astype('object')
test_inputs['discharge_disposition'] = test_inputs['discharge_disposition'].astype('object')


In [9]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [10]:
numeric_columns

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

In [11]:
categorical_columns

['race',
 'gender',
 'age',
 'admission_type',
 'discharge_disposition',
 'admission_source']

In [12]:
feat_eng_columns = ['num_lab_procedures', 'num_procedures']

# Pipeline

In [13]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [14]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col)),
                               ('scaler', StandardScaler())])

In [16]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('trans', my_new_column, feat_eng_columns)    
        ],   
        remainder='passthrough')

#passtrough is an optional step.

# Transform: fit_transform() for TRAIN

In [17]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<6066x65 sparse matrix of type '<class 'numpy.float64'>'
	with 90990 stored elements in Compressed Sparse Row format>

In [18]:
train_x.shape

(6066, 65)

# Tranform: transform() for TEST

In [19]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<2600x65 sparse matrix of type '<class 'numpy.float64'>'
	with 38997 stored elements in Compressed Sparse Row format>

In [20]:
test_x.shape

(2600, 65)

# Calculate the Baseline

In [21]:
# Sci-kit Learn's DummyClassifier automates the entire process performed manually above.
# First, instantiating it with the "most_frequent" strategy. 
# Then, fiting the model. This finds the most frequently (i.e., majority) class (in the training set).

from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
# Then, calling the predict function of the classifier.
# This predicts all values as the majority class.
# In the next step, comparing the predicted values with the actual values to calculate accuracy
# # This is the baseline Train Accuracy

dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.5313221233102539


In [24]:
# We repeat the same steps for the test set
# This is the baseline Test Accuracy

dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5365384615384615


## LOGISTIC REGRESSION MODEL

In [26]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(penalty='none')

log_reg.fit(train_x, train_target)


c:\users\rishabh\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(penalty='none')

In [30]:
log_reg.predict(test_x)
# Create a new DataFrame

predictions = pd.DataFrame(log_reg.predict(test_x), columns=['Predicted'])

# Add the actual to the same DataFrame

predictions['Actual'] = np.array(test_target)

#Predict the train values
train_y_pred = log_reg.predict(train_x)

#Train accuracy
print("Train Accuracy",accuracy_score(train_target, train_y_pred))

#Predict the test values
test_y_pred = log_reg.predict(test_x)

#Test accuracy
print("Test Accuracy",accuracy_score(test_target, test_y_pred))


Train Accuracy 0.6200131882624464
Test Accuracy 0.63


# Train a DT model

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=5)

tree_clf.fit(train_x, train_target)

In [ ]:
from sklearn.metrics import accuracy_score

### Calculate the accuracy

In [ ]:
#Train accuracy:
train_y_pred = tree_clf.predict(train_x)

print(accuracy_score(train_target, train_y_pred))

In [ ]:
#Test accuracy:
test_y_pred = tree_clf.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

In [ ]:
print("baseline accuracy was",baseline_test_acc)

In [ ]:
from sklearn.metrics import confusion_matrix

#Test confusion matrix
confusion_matrix(test_target, test_y_pred)

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import numpy as np

# Assuming tree_clf is trained Decision Tree Classifier
# and train_target is target variable

plt.figure(figsize=(125,50))

tree = plot_tree(tree_clf, 
                 # feature_names=train_inputs.columns.values, # Uncomment if feature names are available
                 class_names=[str(cls) for cls in np.unique(train_target)], 
                 filled=True, 
                 rounded=True, 
                 fontsize=14)
plt.show()


# Train another DT model (with different parameters)

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(
    criterion='entropy',
    splitter='best',
    max_depth=6,
    min_samples_split=4,
    min_samples_leaf=2)


tree_clf.fit(train_x, train_target)



### Calculate the accuracy

In [ ]:
#Train accuracy:
train_y_pred = tree_clf.predict(train_x)

print("Train Accuracy",accuracy_score(train_target, train_y_pred))

In [ ]:
#Test accuracy:
test_y_pred = tree_clf.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

In [ ]:
#Select a random observation

random = test_x[50:51]
random

In [ ]:
# Observe the input variables of the observation

tree_clf.predict_proba(random)

In [ ]:
# Round the probability values

np.round(tree_clf.predict_proba(random),2)

# Optional: try grid search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {'max_depth': randint(low=5, high=20), 
              'min_samples_leaf': randint(low=5, high=20),
             'criterion':['entropy','gini','log_loss'],
             'splitter':['best', 'random']}

tree_gs = RandomizedSearchCV(DecisionTreeClassifier(), param_grid, 
                             n_iter=15, cv=5, verbose=1,
                             scoring='accuracy',
                             return_train_score=True)

tree_gs.fit(train_x, train_target)

In [ ]:
cvres = tree_gs.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

In [ ]:
#Find the best parameter set
tree_gs.best_params_

In [ ]:
tree_gs.best_estimator_

In [ ]:
#Train accuracy:
train_y_pred = tree_gs.best_estimator_.predict(train_x)

print(accuracy_score(train_target, train_y_pred))

In [ ]:
#Test accuracy:
test_y_pred = tree_gs.best_estimator_.predict(test_x)

print(accuracy_score(test_target, test_y_pred))